## 2. Setting up a PyTorch model

Here is a high level overview of the model training process:

- **Objective**: Classify handwritten digits (0-9)
- **Model**: Simple Neural Network using PyTorch
- **Evaluation Metric**: Accuracy
- **Dataset**: MNIST

We'll start with a basic PyTorch implementation to establish a baseline before moving on to more advanced techniques. This will give us a good foundation for understanding the benefits of hyperparameter tuning and distributed training in later sections.

In [3]:
from torchvision.models import resnet18
from torch.optim import Adam
from torch.nn import CrossEntropyLoss


def train_loop_torch(num_epochs: int = 2, batch_size: int = 128, lr: float = 1e-5):
    criterion = CrossEntropyLoss()

    model = resnet18()
    model.conv1 = torch.nn.Conv2d(
        1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
    )
    
    model.to("cuda")
    data_loader = build_data_loader(batch_size)
    optimizer = Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        for images, labels in data_loader:
            images, labels = images.to("cuda"), labels.to("cuda")
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Report the metrics
        print(f"Epoch {epoch}, Loss: {loss}")

We fit the model by submitting it onto a GPU node using Ray Core

In [4]:
train_loop_torch(num_epochs=2)

Epoch 0, Loss: 0.326001912355423
Epoch 1, Loss: 0.20302368700504303


**Can we do any better ?** let's see if we can tune the hyperparameters of our model to get a better loss.

But hyperparameter tuning is a computationally expensive task, and it will take a long time to run sequentially.

[Ray Tune](https://docs.ray.io/en/master/tune/) is a distributed hyperparameter tuning library that can help us speed up the process!